In [1]:
import sys
import os

# Add the project root to the Python path
package_root = os.path.abspath(os.path.join(os.getcwd(), "../"))
print(f"Adding package root to sys.path: {package_root}")
if package_root not in sys.path:
	sys.path.append(package_root)


notebook_dir = os.getcwd()
print(f"Current notebook directory: {notebook_dir}")
# change to the directory to the root of the project
project_root = os.path.abspath(os.path.join(os.getcwd(), "../../"))
print(f"Project root: {project_root}")
os.chdir(project_root)

Adding package root to sys.path: /home/mafzaal/source/lets-talk/py-src
Current notebook directory: /home/mafzaal/source/lets-talk/py-src/notebooks
Project root: /home/mafzaal/source/lets-talk


In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
import lets_talk.utils.blog as blog
import lets_talk.utils.eval as eval

In [4]:
docs = blog.load_blog_posts()
docs = blog.update_document_metadata(docs)
split_docs = blog.split_documents(docs)

100%|██████████| 14/14 [00:00<00:00, 3411.39it/s]

Loaded 14 documents from data/
Split 14 documents into 162 chunks


In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

qa_chat_model = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0,
)

qa_prompt = """\
Given the following context, you must generate questions based on only the provided context.
You are to generate {n_questions} questions which should be provided in the following format:

1. QUESTION #1
2. QUESTION #2
...

Context:
{context}
"""

qa_prompt_template = ChatPromptTemplate.from_template(qa_prompt)
question_generation_chain = qa_prompt_template | qa_chat_model

In [ ]:
import tqdm
import asyncio


def extract_questions(response_text,n_questions):
    # Split the response text into lines
    lines = response_text.strip().split('\n')

    # Extract questions (format: "1. QUESTION")
    extracted_questions = []
    for line in lines:
        line = line.strip()
        if line and any(line.startswith(f"{i}.") for i in range(1, n_questions+1)):
            # Remove the number prefix and get just the question
            question = line.split('.', 1)[1].strip()
            extracted_questions.append(question)

    return extracted_questions

async def create_questions(documents, n_questions, chain):
    question_set = []
    
    for doc in tqdm.tqdm(documents):
        
        context = doc.page_content

        # Generate questions using the question generation chain
        response = await chain.ainvoke({
            "context": context,
            "n_questions": n_questions
        })

        questions = extract_questions(response.content,n_questions)
        
        for i, question in enumerate(questions):
            questions.append({"question":question, "context": context})
    return question_set

In [9]:
context = split_docs[0].page_content
n_questions = 3
response = question_generation_chain.invoke({"context": context, "n_questions": n_questions})
questions = extract_questions(response.content, n_questions)
print("Extracted questions:")
for i, question in enumerate(questions):
    print(f"{i + 1}.  {question}")

Extracted questions:
1.  What is the primary purpose of the Ragas evaluation framework in LLM applications?
2.  Why is it important to have reliable metrics when assessing the performance of LLM-based systems?
3.  What types of applications can benefit from using the Ragas framework for evaluation?


In [ ]:
documents = split_docs[:2]
len(documents)

2

In [16]:
ds = await create_questions(documents=docs, n_questions=3, chain=question_generation_chain)

  0%|          | 0/2 [00:00<?, ?it/s]

: 